In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/claa-18-cmts/Ground Truth 2_P2_Ontology.csv')


In [3]:
df['Category'] =  df['Category'].apply(lambda x:x.strip())
df['Category'] = df['Category'].replace({'claims':'Claims','Account/Settings/Preferences':'account/Settings/Preferences','Other': 'other','Documentation/Forms':'Documentation/forms'})
df = df.dropna()
num_labels = len(set(df['Category']))

label2id, id2label = dict(), dict()
for i, label in enumerate(list(set(df['Category']))):
    label2id[label] = i
    id2label[str(i)] = label

df['Q3-Why'] = df['Q3-Why'].astype(str)
texts = df['Q3-Why'].tolist()

labels = []
for ele in df['Category'].tolist():
    labels.append(label2id[ele])

In [4]:
label2id

{'Address/Address Book': 0,
 'account/Settings/Preferences': 1,
 'Claims': 2,
 'Returns': 3,
 'Printing/Printer': 4,
 'Pricing\\Promo': 5,
 'My Choice': 6,
 'Pickup/Dropoff': 7,
 'Tracking': 8,
 'Delivery': 9,
 'Billing/Payment': 10,
 'Notifications': 11,
 'Quote': 12,
 'Cancel/Void': 13,
 'History': 14,
 'Templates/Batch Shipping': 15,
 'Documentation/forms': 16,
 'other': 17}

In [5]:
id2label

{'0': 'Address/Address Book',
 '1': 'account/Settings/Preferences',
 '2': 'Claims',
 '3': 'Returns',
 '4': 'Printing/Printer',
 '5': 'Pricing\\Promo',
 '6': 'My Choice',
 '7': 'Pickup/Dropoff',
 '8': 'Tracking',
 '9': 'Delivery',
 '10': 'Billing/Payment',
 '11': 'Notifications',
 '12': 'Quote',
 '13': 'Cancel/Void',
 '14': 'History',
 '15': 'Templates/Batch Shipping',
 '16': 'Documentation/forms',
 '17': 'other'}

In [6]:
len(df)

9982

In [7]:
df.columns

Index(['Q3-Why', 'Category'], dtype='object')

In [8]:
from sklearn.model_selection import train_test_split

# In the first step we will split the data in training and remaining dataset
x_train, X_rem, y_train, y_rem = train_test_split(df['Q3-Why'],df['Category'], train_size=0.7,random_state=42)

# train, valid, test -> 70, 10, 20
x_valid, x_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.666,random_state=42)

print(x_train.shape), print(y_train.shape)
print(x_valid.shape), print(y_valid.shape)
print(x_test.shape), print(y_test.shape)
print(type(x_train))


(6987,)
(6987,)
(1000,)
(1000,)
(1995,)
(1995,)
<class 'pandas.core.series.Series'>


In [ ]:
# df = pd.DataFrame(list(zip(x_train, y_train)), columns = ['Q3-Why', 'Category'])

In [9]:
df2 = pd.DataFrame(list(zip(x_valid, y_valid)),columns =['Q3-Why', 'Category'])

In [10]:
df3 = pd.DataFrame(list(zip(x_test, y_test)),columns =['Q3-Why', 'Category'])


In [11]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [12]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [14]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [15]:
# !pip install datasets

In [16]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

In [18]:
str_to_idx = label2id

In [19]:
x_train, y_train = df['Q3-Why'],df['Category']
x_valid, y_valid = df2['Q3-Why'],df2['Category']
x_test, y_test = df3['Q3-Why'],df3['Category']

# {'Cost', 'Operational', 'Others', 'Performance', 'Sales', 'Usablity'}

y_train = [str_to_idx[c] for c in y_train]

y_valid = [str_to_idx[c] for c in y_valid]


y_test = [str_to_idx[c] for c in y_test]

In [20]:
df = pd.DataFrame(list(zip(x_train, y_train)), columns = ['Q3-Why', 'Category'])
df2 = pd.DataFrame(list(zip(x_valid, y_valid)), columns = ['Q3-Why', 'Category'])
df3 = pd.DataFrame(list(zip(x_test, y_test)), columns = ['Q3-Why', 'Category'])

In [21]:
hg_dataset_train = Dataset(pa.Table.from_pandas(df))
hg_dataset_valid = Dataset(pa.Table.from_pandas(df2))
hg_dataset_test = Dataset(pa.Table.from_pandas(df3))

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [22]:
import datasets
cmts = datasets.DatasetDict({"train":hg_dataset_train,"valid":hg_dataset_valid,"test":hg_dataset_test})

In [23]:
cmts


DatasetDict({
    train: Dataset({
        features: ['Q3-Why', 'Category'],
        num_rows: 9982
    })
    valid: Dataset({
        features: ['Q3-Why', 'Category'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['Q3-Why', 'Category'],
        num_rows: 1995
    })
})

In [24]:
def tokenize(batch):
    return tokenizer(batch["Q3-Why"], padding=True, truncation=True)

In [25]:
cmts_encoded = cmts.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
np.object = object    

In [28]:
cmts_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'Category'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 4

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(cmts_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
val_dataset = tf.data.Dataset.from_tensor_slices(cmts_encoded['valid'][:])
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(cmts_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [29]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[ 101, 2498, 2001, ...,    0,    0,    0],
       [ 101, 1996, 2578, ...,    0,    0,    0],
       [ 101, 1045, 2525, ...,    0,    0,    0],
       [ 101, 2004,  102, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([17  9 17 17], shape=(4,), dtype=int64)


In [30]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [31]:
classifier = BERTForClassification(model, num_classes=18)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [32]:
history = classifier.fit(
    train_dataset,
    epochs=4
)


Epoch 1/4
2496/2496 [==============================] - 1360s 524ms/step - loss: 1.1316 - accuracy: 0.6863
Epoch 2/4
2496/2496 [==============================] - 1304s 522ms/step - loss: 0.6744 - accuracy: 0.7996
Epoch 3/4
2496/2496 [==============================] - 1304s 522ms/step - loss: 0.5395 - accuracy: 0.8314
Epoch 4/4
2496/2496 [==============================] - 1304s 523ms/step - loss: 0.4199 - accuracy: 0.8612


In [33]:
df4= pd.read_csv("/kaggle/input/deploy-dataset/Deploy_dashboard_final_7_13.csv")

In [34]:
len(df4)

39569

In [35]:
df4.columns

Index(['Start Date (+00:00 GMT)', 'Survey Date (+00:00 GMT)', 'NPS Group',
       'Q1 - How likely are you to recommend UPS to a friend or colleague?',
       'Q2 - To what extent did your most recent [journey experience] experience influen...',
       'Q3 - Why', 'Q4 - What'],
      dtype='object')

In [39]:
df5 = df4[['Q3 - Why']]
df5['Q3 - Why'] = df5['Q3 - Why'].astype(str)
hg_dataset_deploy = Dataset(pa.Table.from_pandas(df5))
cmts_deploy = datasets.DatasetDict({"deploy":hg_dataset_deploy})

def tokenize(batch):
    return tokenizer(batch["Q3 - Why"], padding=True, truncation=True)

cmts_deploy_encoded = cmts_deploy.map(tokenize, batched=True, batch_size=None)

cmts_deploy_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 4

# converting train split of `emotions_encoded` to tensorflow format
deploy_dataset = tf.data.Dataset.from_tensor_slices(cmts_deploy_encoded['deploy'][:])
# set batch_size and shuffle
deploy_dataset = deploy_dataset.batch(BATCH_SIZE).shuffle(1000)

/tmp/ipykernel_47/3834172596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Q3 - Why'] = df5['Q3 - Why'].astype(str)
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
pred_test = classifier.predict(deploy_dataset)
pred_test_array = np.array(pred_test)
np.save("Q3M2.npy", pred_test_array)

9893/9893 [==============================] - 1721s 174ms/step


In [41]:
df5 = df4[['Q4 - What']]
df5['Q4 - What'] = df5['Q4 - What'].astype(str)
hg_dataset_deploy = Dataset(pa.Table.from_pandas(df5))
cmts_deploy = datasets.DatasetDict({"deploy":hg_dataset_deploy})

def tokenize(batch):
    return tokenizer(batch["Q4 - What"], padding=True, truncation=True)

cmts_deploy_encoded = cmts_deploy.map(tokenize, batched=True, batch_size=None)

cmts_deploy_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 4

# converting train split of `emotions_encoded` to tensorflow format
deploy_dataset = tf.data.Dataset.from_tensor_slices(cmts_deploy_encoded['deploy'][:])
# set batch_size and shuffle
deploy_dataset = deploy_dataset.batch(BATCH_SIZE).shuffle(1000)

/tmp/ipykernel_47/3726025516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Q4 - What'] = df5['Q4 - What'].astype(str)
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
pred_test = classifier.predict(deploy_dataset)
pred_test_array = np.array(pred_test)
np.save("Q4M2.npy", pred_test_array)

9893/9893 [==============================] - 1714s 173ms/step


In [ ]:


loaded_array = np.load("test_pred.npy")
print(loaded_array)

In [ ]:
pred_test = classifier.predict(test_dataset)

In [ ]:
pred_val = classifier.predict(val_dataset)
pred_val_array = np.array(pred_val)
np.save("val_pred.npy", pred_val_array)

loaded_array_val = np.load("val_pred.npy")
print(loaded_array_val)

In [ ]:
classifier.evaluate(test_dataset)


In [ ]:
classifier.evaluate(val_dataset)


In [ ]:
pred_test_array[0]

In [ ]:
pred_test_res = np.argmax(pred_test_array, axis=1)
pred_test_res = list(pred_test_res)
actual_test = y_test

mismatch = 0
for i in range(len(actual_test)):
    if pred_test_res[i] !=  actual_test[i]:
        mismatch += 1
print(mismatch)

In [ ]:
len(y_test)

In [ ]:
print(f"no of validation datas {len(y_valid)}")
pred_valid_res = np.argmax(pred_val_array, axis=1)
pred_valid_res = list(pred_valid_res)
actual_valid = y_valid

mismatch_v = 0
for i in range(len(actual_valid)):
    if pred_valid_res[i] !=  actual_valid[i]:
        mismatch_v += 1
print(mismatch_v)

In [ ]:
saved_model_path = "/kaggle/working/new"
classifier.save(saved_model_path)


In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)


In [ ]:
saved_model_path

In [ ]:
!zip -r file.zip /kaggle/working/imdb


In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

In [ ]:
zip_dir()